In [1]:
import torch


In [2]:
import numpy as np

ar = np.array([1., 2., 3., 4., 5.])

ar[2] = np.nan

np.nan_to_num(ar, nan=np.inf, copy=False)
ar

array([ 1.,  2., inf,  4.,  5.])

In [3]:
# timeit

N = 100
M = 4
v = torch.randn(N)
v
K = torch.randn(N, N).abs()

A = torch.randn(N, M)

@torch.jit.script
def f1(K,A):
    f_hat = K @ A
    f_hat_K_tch_inv = torch.linalg.solve(K, f_hat)
    return torch.trace(f_hat.T @ f_hat_K_tch_inv)

@torch.jit.script
def f2(K,A):
    f_hat = K @ A
    return torch.trace(f_hat.T @ torch.inverse(K) @ f_hat)

def f3(K,A):
    f_hat = K @ A
    f_hat_K_tch_inv = torch.linalg.solve(K, f_hat)
    return torch.trace(f_hat.T @ f_hat_K_tch_inv)

def f4(K,A):
    f_hat = K @ A
    return torch.trace(f_hat.T @ torch.inverse(K) @ f_hat)
f1(K,A)

# %timeit f1(K,A)
# %timeit f2(K,A)
# %timeit f3(K,A)
# %timeit f4(K,A)

tensor(380.4066)

In [4]:

import torch
from stpy.continuous_processes.gauss_procc import GaussianProcess
from stpy.helpers.helper import interval


# Parameters
N = 5
n = 256
d = 1
eps = 0.01
s = 1
gamma = 0.1

# Set random seed
torch.manual_seed(1)

# Generate random data
x = torch.rand(N, d).double() * 2 - 1
xtest = torch.from_numpy(interval(n, d, L_infinity_ball=1))

# True Gaussian process
GP_true = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d)
ytest = GP_true.sample(xtest)
GP_true.fit_gp(xtest, ytest)
y = GP_true.mean(x).clone()

# New data points
xnew = x[0, :].view(1, 1) + eps
ynew = y[0, 0].view(1, 1) + 1

# Combined data
x2 = torch.vstack([x, xnew])
y2 = torch.vstack([y, ynew])

# Initialize models
gp_cvx = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='huber', huber_delta=1.5)
# gp_torch = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='huber_torch', huber_delta=1.5)
gp_st = GaussianProcess(gamma=gamma, kernel_name="squared_exponential", d=d, loss='studentT', lam=0.005)

gp_st.fit_gp(x2, y2)
from torch.profiler import profile, record_function, ProfilerActivity 

with profile(activities=[ProfilerActivity.CPU], record_shapes=False) as prof:
    # Fit models
    gp_st.optimize_params(type="bandwidth", restarts=1, verbose=True, optimizer='pytorch-minimize', scale=1., weight=1., maxiter=1)

# Fit models
# %timeit gp_cvx.fit_gp(x2, y2)
# %timeit gp_torch.fit_gp(x2, y2)

# mean_cvx = gp_cvx.mean(xtest)
# mean_torch = gp_torch.mean(xtest)
mean_st = gp_st.mean(xtest)


c:\Users\queim\micromambaenv\envs\stpy39\lib\site-packages\torch\jit\_script.py:1138: UserWarning: Warning: monkeytype is not installed. Please install https://github.com/Instagram/MonkeyType to enable Profile-Directed Typing in TorchScript. Refer to https://github.com/Instagram/MonkeyType/blob/master/README.rst to install MonkeyType. 
  warnings.warn(


tensor(-15.0465, dtype=torch.float64)
Restart 0


c:\Users\queim\micromambaenv\envs\stpy39\lib\site-packages\torch\jit\_script.py:1138: UserWarning: Warning: monkeytype is not installed. Please install https://github.com/Instagram/MonkeyType to enable Profile-Directed Typing in TorchScript. Refer to https://github.com/Instagram/MonkeyType/blob/master/README.rst to install MonkeyType. 
  warnings.warn(


tensor(-7.3259, dtype=torch.float64)
initial fval: 194.6078
tensor(-7.0117, dtype=torch.float64)
tensor(-7.3259, dtype=torch.float64)
tensor(-7.1986, dtype=torch.float64)
tensor(-7.3260, dtype=torch.float64)
tensor(-7.0083, dtype=torch.float64)
tensor(-7.0109, dtype=torch.float64)
iter   1 - fval: 192.5192
Maximum number of iterations has been exceeded.
         Current function value: 192.519225
         Iterations: 1
         Function evaluations: 7
0 Cost: 192.519, Params: 4.558496
Best index: 0
Kernel description:

	kernel: squared_exponential
	operation: -
	kappa=1.0
	group=[0]
	offset=0.0
	gamma=tensor([4.5585], dtype=torch.float64)
lambda=0.001
tensor(4.7338, dtype=torch.float64)
tensor(4.7453, dtype=torch.float64)


In [10]:
prof.export_chrome_trace("trace.json")

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=100))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                          Optimizer.step#Minimizer.step        44.22%        3.315s        98.08%        7.353s     919.067ms             8  
                                              aten::dot         6.11%     458.381ms         7.86%     589.061ms       3.897us        151173  
                                           aten::select         6.49%     486.507ms         7.59%     569.136ms       2.586us        220068  
                                              aten::mul         5.75%     430.789ms         5.96%     446.719ms       2.361us        189192  
      

In [6]:

import matplotlib.pyplot as plt
 
plt.plot(xtest, mean_cvx, label='cvx')
# plt.plot(xtest, mean_torch, label='torch')
plt.plot(x, y, 'o', label='data')
plt.plot(xnew, ynew, 'o', label='new data')

plt.legend()

NameError: name 'mean_cvx' is not defined